# <b><u>Hand gesture detection</u></b>
This project focuses on detecting hand textures as an initial step toward implementing hand gesture-based control for hardware systems. The solution utilizes MediaPipe and OpenCV to detect and process hand landmarks in real time. These landmarks will serve as the foundation for recognizing gestures and triggering corresponding actions in future development stages.

### 1. Hand Detection and Gesture-Based Cropping

In [2]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

with mp_hands.Hands(static_image_mode=False,
                    max_num_hands=1,
                    min_detection_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # Flip horizontally

        if not ret:
            break

        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(img_rgb)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Get frame size
                h, w, _ = frame.shape

                # Get coordinates and clamp them safely
                x_coords = [int(lm.x * w) for lm in hand_landmarks.landmark]
                y_coords = [int(lm.y * h) for lm in hand_landmarks.landmark]
                x_min, x_max = max(min(x_coords), 0), min(max(x_coords), w)
                y_min, y_max = max(min(y_coords), 0), min(max(y_coords), h)

                # Only crop if valid area
                if x_max > x_min and y_max > y_min:
                    hand_crop = frame[y_min:y_max, x_min:x_max]
                    cv2.imshow("Hand Crop", hand_crop)

        cv2.imshow("Hand Tracking", frame)
        key = cv2.waitKey(1)
        if key == ord('q') or key == 27:  # 'q' or ESC key
            break

cap.release()
cv2.destroyAllWindows()



### 2. View hand landmarks with names

In [3]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_hands_module = mp.solutions.hands  # For accessing HandLandmark enum

cap = cv2.VideoCapture(0)

with mp_hands.Hands(static_image_mode=False,
                    max_num_hands=1,
                    min_detection_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # Flip horizontally

        if not ret:
            break

        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(img_rgb)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                h, w, _ = frame.shape

                for idx, landmark in enumerate(hand_landmarks.landmark):
                    cx, cy = int(landmark.x * w), int(landmark.y * h)

                    # Draw landmark index or name
                    landmark_name = mp_hands_module.HandLandmark(idx).name
                    cv2.putText(frame, landmark_name, (cx + 5, cy - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)

        cv2.imshow("Hand Tracking", frame)
        key = cv2.waitKey(1)
        if key == ord('q') or key == 27:
            break

cap.release()
cv2.destroyAllWindows()

### 3. Measure the landmarks distanse using Z-depth values and draw a circle
I am going to measure the distance beetween thumb tip and index finger tip

<b><u>1. Use one hand</u></b>

In [15]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_hands_module = mp.solutions.hands  # For accessing HandLandmark enum

# Initialize variables for tracking min and max distance
distance_history = []

cap = cv2.VideoCapture(0)

with mp_hands.Hands(static_image_mode=False,
                    max_num_hands=1,
                    min_detection_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # Flip horizontally

        if not ret:
            break

        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(img_rgb)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                h, w, _ = frame.shape

                # Get coordinates of thumb tip and index finger tip
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

                # Get 3D coordinates for depth (z values)
                z_thumb = thumb_tip.z
                z_index = index_tip.z

                # Calculate Euclidean distance (ignoring z for now)
                x1, y1 = int(thumb_tip.x * w), int(thumb_tip.y * h)
                x2, y2 = int(index_tip.x * w), int(index_tip.y * h)

                # Calculate the Euclidean distance between the thumb and index fingertip
                distance = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

                # Incorporate z-depth into the distance calculation
                # Use the difference in the z-values to adjust the final distance
                z_distance = abs(z_thumb - z_index)
                adjusted_distance = distance + z_distance * 100  # Multiply z difference to adjust effect

                # Add to distance history
                distance_history.append(adjusted_distance)

                # Keep only the min and max distances in the array
                if len(distance_history) > 100:  # Limit history length to 100 values
                    distance_history.pop(0)  # Remove the oldest value

                min_distance = min(distance_history)
                max_distance = max(distance_history)

                # Show the min and max distance
                cv2.putText(frame, f"Min Dist: {min_distance:.2f}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(frame, f"Max Dist: {max_distance:.2f}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


                # Draw a circle using landmarks distanse between 
                radius = int(adjusted_distance)
                
                # Calculate a random position for the circle (ensuring it's within the frame)
                center_x = w//2
                center_y = h//2
                
                # Draw the circle on the frame
                color = (0, 255, 0)  # Green color for the circle
                thickness = 2  # Thickness of the circle outline
                cv2.circle(frame, (center_x, center_y), radius, color, thickness)

        cv2.imshow("Hand Tracking", frame)
        key = cv2.waitKey(1)
        if key == ord('q') or key == 27:  # 'q' or ESC key
            break
        if key == ord('s'):  # 'print the min and max so far
            print(f'Min is {min_distance:.2f} and Max is {max_distance:.2f}')

cap.release()
cv2.destroyAllWindows()

<b><u>2. Use both hands and draw two circles</u></b>

In [1]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

distance_history = []
hand_distances = [0, 0]  # To track distances of two hands separately

cap = cv2.VideoCapture(0)

with mp_hands.Hands(static_image_mode=False,
                    max_num_hands=2,
                    min_detection_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        # frame = cv2.resize(frame, (800, 500))

        if not ret:
            break

        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(img_rgb)

        h, w, _ = frame.shape

        if result.multi_hand_landmarks:
            for hand_index, hand_landmarks in enumerate(result.multi_hand_landmarks):
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

                x1, y1 = int(thumb_tip.x * w), int(thumb_tip.y * h)
                x2, y2 = int(index_tip.x * w), int(index_tip.y * h)

                z_thumb = thumb_tip.z
                z_index = index_tip.z

                distance = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
                z_distance = abs(z_thumb - z_index)
                adjusted_distance = distance + z_distance * 100

                # Save distance for display
                if hand_index < 2:
                    hand_distances[hand_index] = adjusted_distance

                # Save to history
                distance_history.append(adjusted_distance)
                if len(distance_history) > 100:
                    distance_history.pop(0)

                min_distance = min(distance_history)
                max_distance = max(distance_history)

                # Draw colored circle offset
                offset = 150
                center_y = h // 2

                if hand_index == 0:
                    center_x = w // 2 - offset
                    color = (0, 255, 0)  # Green
                    tcolor = (0, 255, 0)
                else:
                    center_x = w // 2 + offset
                    color = (0, 0, 255)  # Red
                    tcolor = (0, 0, 255)

                radius = int(adjusted_distance)
                cv2.circle(frame, (center_x, center_y), radius, color, 2)

            # Display text info
            cv2.putText(frame, f"Min Dist: {min_distance:.2f}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(frame, f"Max Dist: {max_distance:.2f}", (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            # Show both current distances
            dist_text = f"Current Dist 1st: {hand_distances[0]:.2f} | 2nd: {hand_distances[1]:.2f}"
            cv2.putText(frame, dist_text, (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

        cv2.imshow("Hand Tracking", frame)
        key = cv2.waitKey(1)
        if key == ord('q') or key == 27:
            break
        if key == ord('s'):
            print(f'Min distance sofar is {min_distance:.2f} and Max is {max_distance:.2f}')
            print(f'Current Dist 1st: {hand_distances[0]:.2f} | 2nd: {hand_distances[1]:.2f}')

cap.release()
cv2.destroyAllWindows()

Min distance sofar is 19.88 and Max is 163.02
Current Dist 1st: 91.60 | 2nd: 93.68
